In [1]:
with open(r"C:\Users\Niv-Levi\Desktop\לימודים\שנה ג'\כריית נתונים בפייתון\Matala3\dests.txt", "r") as hand:
    cities = []
    for line in hand:
        line = line.rstrip()
        words = line.split()
        for i in words:
            cities.append(i)

In [2]:
import requests

#####Get Distance_km and Duration#############

# Set up the API endpoint and parameters
distancematrix_url = 'https://maps.googleapis.com/maps/api/distancematrix/json'

data_list = []
params = {
        'origins': "Tel Aviv",
        'destinations': "|".join(cities), #Its better to request the api 1 time * 5 cities instead 5 time * 1 city.
        'key': "XXXXXXXXXXXXXXXXXX"}


# Send HTTP request 
try:
    response = requests.get(distancematrix_url, params=params)
    if response.status_code != 200:
        print("HTTP error", response.status_code)
    else : 
        try : 
            data = response.json()
        except:
            print("Response not in valid JSON format")
except:
    print("Something went wrong with rquests.get")

for city in range(len(cities)) :      
        
    # Retrive the necessary data: 
    my_dict = {
    'Target': data['destination_addresses'][city],
    'Distance_km': data['rows'][0]['elements'][city]['distance']['text'], 
    'Duration': data['rows'][0]['elements'][city]['duration']['value'] / 60 #If you want to extract the duration by minutes.
    }
    data_list.append(my_dict)

In [3]:
data

{'destination_addresses': ['İstanbul, Türkiye',
  'Amsterdam, Netherlands',
  'Valletta, Malta',
  'Basel, Switzerland',
  'Doha, Qatar'],
 'origin_addresses': ['Tel Aviv-Yafo, Israel'],
 'rows': [{'elements': [{'distance': {'text': '1,815 km', 'value': 1815227},
     'duration': {'text': '21 hours 3 mins', 'value': 75772},
     'status': 'OK'},
    {'distance': {'text': '4,533 km', 'value': 4532541},
     'duration': {'text': '2 days 0 hours', 'value': 173008},
     'status': 'OK'},
    {'distance': {'text': '3,793 km', 'value': 3792883},
     'duration': {'text': '2 days 3 hours', 'value': 183056},
     'status': 'OK'},
    {'distance': {'text': '4,093 km', 'value': 4092805},
     'duration': {'text': '1 day 20 hours', 'value': 158547},
     'status': 'OK'},
    {'distance': {'text': '2,164 km', 'value': 2164477},
     'duration': {'text': '22 hours 39 mins', 'value': 81527},
     'status': 'OK'}]}],
 'status': 'OK'}

In [4]:
data_list

[{'Target': 'İstanbul, Türkiye',
  'Distance_km': '1,815 km',
  'Duration': 1262.8666666666666},
 {'Target': 'Amsterdam, Netherlands',
  'Distance_km': '4,533 km',
  'Duration': 2883.4666666666667},
 {'Target': 'Valletta, Malta',
  'Distance_km': '3,793 km',
  'Duration': 3050.9333333333334},
 {'Target': 'Basel, Switzerland',
  'Distance_km': '4,093 km',
  'Duration': 2642.45},
 {'Target': 'Doha, Qatar',
  'Distance_km': '2,164 km',
  'Duration': 1358.7833333333333}]

In [5]:
######### Get Logitude and Latitude#############
### Geocode doesn't return joined request sorted a-z so you have to request for each city.

# Set up the API 
geocode_url = 'https://maps.googleapis.com/maps/api/geocode/json'

for index in range(len(cities)):
    params = {
        'address': cities[index],
        'key': "XXXXXXXXXXXXXXX"
    }

    try:
        response = requests.get(geocode_url, params=params)
        if response.status_code != 200:
            print("HTTP error", response.status_code)
        else : 
            try : 
                data2 = response.json()
            except:
                print("Response not in valid JSON format")
                continue
    except:
        print("Something went wrong with rquests.get")
        continue
        
    #for unrecognize cities
    if data2['status'] != 'OK':
        continue
    
    # Retrive the necessary data to the exiciting dicts: 
    data_list[index]["Latutude"] = data2['results'][0]['geometry']['location']['lat']
    data_list[index]["Longitude"] = data2['results'][0]['geometry']['location']['lng']

In [12]:
data_list

[{'Target': 'İstanbul, Türkiye',
  'Distance_km': '1,815 km',
  'Duration': 1262.8666666666666,
  'Latutude': 41.0082376,
  'Longitude': 28.9783589},
 {'Target': 'Amsterdam, Netherlands',
  'Distance_km': '4,533 km',
  'Duration': 2883.4666666666667,
  'Latutude': 52.3675734,
  'Longitude': 4.9041389},
 {'Target': 'Valletta, Malta',
  'Distance_km': '3,793 km',
  'Duration': 3050.9333333333334,
  'Latutude': 35.8992375,
  'Longitude': 14.5140996},
 {'Target': 'Basel, Switzerland',
  'Distance_km': '4,093 km',
  'Duration': 2642.45,
  'Latutude': 47.5595986,
  'Longitude': 7.5885761},
 {'Target': 'Doha, Qatar',
  'Distance_km': '2,164 km',
  'Duration': 1358.7833333333333,
  'Latutude': 25.2854473,
  'Longitude': 51.53103979999999}]

In [13]:
import pandas as pd
df = pd.DataFrame(data_list)

In [14]:
def calculate_time(total_minutes):
    hours = round(total_minutes // 60)
    minutes = round(total_minutes % 60)
    time_string = "{:02d}:{:02d}".format(hours, minutes)
    return time_string
df["Duration"] = df["Duration"].apply(calculate_time)
df

,Target,Distance_km,Duration,Latutude,Longitude
0,"İstanbul, Türkiye","1,815 km",21:03,41.008238,28.978359
1,"Amsterdam, Netherlands","4,533 km",48:03,52.367573,4.904139
2,"Valletta, Malta","3,793 km",50:51,35.899237,14.514100
3,"Basel, Switzerland","4,093 km",44:02,47.559599,7.588576
4,"Doha, Qatar","2,164 km",22:39,25.285447,51.531040


In [15]:
import re
df['Distance_km'] = df['Distance_km'].str.extract(r'(\d+,\d+)')
df['Distance_km'] = df['Distance_km'].str.replace(",","").astype(int)

In [16]:
df.sort_values('Distance_km', ascending=False).head(3)

,Target,Distance_km,Duration,Latutude,Longitude
1,"Amsterdam, Netherlands",4533,48:03,52.367573,4.904139
3,"Basel, Switzerland",4093,44:02,47.559599,7.588576
2,"Valletta, Malta",3793,50:51,35.899237,14.514100
